In [8]:
import usb.core
import usb.util

# Find all USB devices
dev = usb.core.find(find_all=True)


In [9]:
dev

<generator object find.<locals>.device_iter at 0x000002ABD7137F20>

In [10]:

# Iterate over all devices
for cfg in dev:
    print(cfg)
    # Get the device's vendor and product IDs
    vid = cfg.idVendor
    pid = cfg.idProduct

    # Check if the device is the one you're looking for
    if vid == 0x067B and pid == 0x2303:
        # Set the configuration
        cfg.set_configuration()

        # Claim the interface
        intf = cfg[(0,0)]

        # Define the data to send
        data = b'Hello, world!'

        # Send the data through the USB cable
        intf.write(0x01, data)

        # Modulate the laser diode using the transistor circuit
        while True:
            # Turn the laser diode on
            intf.write(0x01, b'\x01')
            time.sleep(5)
            # Turn the laser diode off
            intf.write(0x01, b'\x00')
            time.sleep(1)

DEVICE ID 067b:2303 on Bus 000 Address 001 =================
 bLength                :   0x12 (18 bytes)
 bDescriptorType        :    0x1 Device
 bcdUSB                 :  0x110 USB 1.1
 bDeviceClass           :    0x0 Specified at interface
 bDeviceSubClass        :    0x0
 bDeviceProtocol        :    0x0
 bMaxPacketSize0        :   0x40 (64 bytes)
 idVendor               : 0x067b
 idProduct              : 0x2303
 bcdDevice              :  0x400 Device 4.0
 iManufacturer          :    0x1 Prolific Technology Inc.
 iProduct               :    0x2 USB-Serial Controller
 iSerialNumber          :    0x0 
 bNumConfigurations     :    0x1
  CONFIGURATION 1: 100 mA ==================================
   bLength              :    0x9 (9 bytes)
   bDescriptorType      :    0x2 Configuration
   wTotalLength         :   0x27 (39 bytes)
   bNumInterfaces       :    0x1
   bConfigurationValue  :    0x1
   iConfiguration       :    0x0 
   bmAttributes         :   0x80 Bus Powered
   bMaxPower      

TypeError: '>=' not supported between instances of 'tuple' and 'int'

In [ ]:
import usb.core
import usb.util

# Define the USB vendor and product ID
VENDOR_ID = 0x067B  # Replace with your USB-Serial converter's vendor ID
PRODUCT_ID = 0x2303  # Replace with your USB-Serial converter's product ID

def find_device():
    # Find the USB device
    device = usb.core.find(idVendor=VENDOR_ID, idProduct=PRODUCT_ID)
    
    if device is None:
        raise ValueError("Device not found")
    
    return device

def configure_device(device):
    # Set the active configuration
    device.set_configuration()
    
    # Claim the interface
    usb.util.claim_interface(device, 0)

def send_data(device, data):
    # Sending data to the device
    # Endpoint addresses can vary. Replace 0x02 with the actual OUT endpoint address of your device.
    endpoint_out = 0x02  # Typically, OUT endpoints have even numbers
    try:
        device.write(endpoint_out, data)
        print(f"Data sent: {data}")
    except usb.core.USBError as e:
        print(f"Failed to send data: {str(e)}")

def main():
    # Find and configure the device
    device = find_device()
    configure_device(device)
    
    # Data to send to light up the laser diode
    # This data should be determined based on your device's protocol
    # For example, sending a command to toggle the TX pin
    data = b'\x000000111111'  # Replace with the actual command/data to control the laser diode

    # Send the data
    send_data(device, data)
    
    # Release the device
    usb.util.release_interface(device, 0)
    usb.util.dispose_resources(device)

if __name__ == "__main__":
    main()


In [ ]:
import win32api

def get_connected_usb_devices():
    devices = win32api.GetLogicalDriveStrings().split('\000')[:-1]
    usb_devices = []
    for device in devices:
        if device.startswith('USB'):
            usb_devices.append(device)
    return usb_devices

print(get_connected_usb_devices())

In [ ]:
import usb.core
import usb.util
import time

# Define the USB vendor and product ID
VENDOR_ID = 0x067b  # Replace with your USB-Serial converter's vendor ID
PRODUCT_ID = 0x2303  # Replace with your USB-Serial converter's product ID

def find_device():
    # Find the USB device
    device = usb.core.find(idVendor=VENDOR_ID, idProduct=PRODUCT_ID)
    
    if device is None:
        raise ValueError("Device not found")
    
    return device

def configure_device(device):
    # Set the active configuration
    device.set_configuration()
    
    # Claim the interface
    usb.util.claim_interface(device, 0)

def send_data(device, data):
    # Sending data to the device
    # Endpoint addresses can vary. Replace 0x02 with the actual OUT endpoint address of your device.
    endpoint_out = 0x02  # Typically, OUT endpoints have even numbers
    try:
        device.write(endpoint_out, data)
        print(f"Data sent: {data}")
    except usb.core.USBError as e:
        print(f"Failed to send data: {str(e)}")

def main():
    # Find and configure the device
    device = find_device()
    configure_device(device)
    
    # Example data to send to light up the laser diode
    # This data should be determined based on your device's protocol
    # For example, sending a command to toggle the TX pin
    on_command = b'\x01'  # Replace with the actual command to turn on the laser diode
    off_command = b'\x00'  # Replace with the actual command to turn off the laser diode

    # Send the data to turn on the laser diode
    send_data(device, on_command)
    time.sleep(1)  # Keep the laser diode on for 1 second
    
    # Send the data to turn off the laser diode
    send_data(device, off_command)
    
    # Release the device
    usb.util.release_interface(device, 0)
    usb.util.dispose_resources(device)

if __name__ == "__main__":
    main()


In [ ]:
import serial
import time

def find_available_ports():
    # List available ports
    available_ports = [f'COM{i}' for i in range(1, 257)]
    active_ports = []
    for port in available_ports:
        try:
            s = serial.Serial(port)
            s.close()
            active_ports.append(port)
        except (OSError, serial.SerialException):
            pass
    return active_ports

def open_serial_port(com_port):
    try:
        ser = serial.Serial(com_port, baudrate=9600, timeout=1)
        print(f"Opened {com_port} successfully.")
        return ser
    except serial.SerialException as e:
        print(f"Failed to open {com_port}: {e}")
        return None

def send_command(ser, command):
    if ser:
        ser.write(command)
        print(f"Data sent: {command}")
    else:
        print("Serial port not available")

def main():
    # Find available ports
    active_ports = find_available_ports()
    print(f"Available ports: {active_ports}")
    
    # Use the first available port (update as necessary)
    if active_ports:
        com_port = active_ports[0]  # For example, 'COM3'
    else:
        print("No available COM ports found.")
        return
    
    ser = open_serial_port(com_port)
    
    if ser:
        try:
            on_command = b'\x01'  # Replace with the actual command to turn on the laser diode
            off_command = b'\x00'  # Replace with the actual command to turn off the laser diode

            print("Turning on the laser diode...")
            send_command(ser, on_command)
            time.sleep(1)  # Keep the laser diode on for 1 second

            print("Turning off the laser diode...")
            send_command(ser, off_command)
            time.sleep(1)  # Delay before finishing
        finally:
            ser.close()
            print(f"Closed {com_port}")

if __name__ == "__main__":
    main()


In [ ]:
import usb.core
import usb.util
import time

# Find the USB device
dev = usb.core.find(idVendor=0x067B, idProduct=0x2303)
# Set the configuration
dev.set_configuration()

# Get the first interface
intf = dev[0][(0,0)]

# Get the endpoints
ep_out = intf[1]  # Endpoint 0x2: Bulk OUT
ep_in = intf[2]  # Endpoint 0x83: Bulk IN

while True:
    print("Setting laser to ON")
    ep_out.write(b'\x01')
    time.sleep(0.5)  # 500ms delay
    print("Setting laser to OFF")
    ep_out.write(b'\x00')
    time.sleep(0.5)  # 500ms delay

In [ ]:
import usb.core
import usb.util
import time

# Find the USB device
dev = usb.core.find(idVendor=0x067b, idProduct=0x2303)

# Set the configuration
dev.set_configuration()

# Get the interface
intf = dev[0][(0,0)]

# Get the endpoint
ep_out = intf[1]

while True:
    # Set the LED on
    ep_out.write(b'\x01')
    time.sleep(0.5)  # 500ms delay

    # Set the LED off
    ep_out.write(b'\x00')
    time.sleep(0.5)  # 500ms delay

In [6]:
import usb.core
import usb.util
import time

# Find the USB device
vendor_id = 0x067b
product_id = 0x2303
device = usb.core.find(idVendor=vendor_id, idProduct=product_id)

if device is None:
    raise ValueError("Device not found")

# Set the active configuration. With no arguments, the first configuration will be the active one
device.set_configuration()

# Get the active configuration
cfg = device.get_active_configuration()
interface_number = cfg[(0, 0)].bInterfaceNumber

# Claim the interface
usb.util.claim_interface(device, interface_number)

# Find the Bulk OUT endpoint (0x02)
endpoint_out = usb.util.find_descriptor(
    cfg[(0, 0)],
    custom_match=lambda e: usb.util.endpoint_direction(e.bEndpointAddress) == usb.util.ENDPOINT_OUT
)

assert endpoint_out.bEndpointAddress == 0x02

def blink_laser(duration_on, duration_off, iterations):
    for _ in range(iterations):
        endpoint_out.write(b'0')  # Send '1' to turn on the laser (turn on the transistor)
        time.sleep(duration_on)  # Keep the laser on for the specified duration
        endpoint_out.write(b'1')  # Send '0' to turn off the laser (turn off the transistor)
        time.sleep(duration_off)  # Keep the laser off for the specified duration

try:
    blink_laser(5, 0.5, 10)  # Blink the laser 10 times with 0.5 seconds on and off
finally:
    # Release the interface and resources
    usb.util.release_interface(device, interface_number)
    usb.util.dispose_resources(device)


KeyboardInterrupt: 

In [13]:
import usb.core
import usb.util
import time

# Find the device
dev = usb.core.find(idVendor=0x067b, idProduct=0x2303)

# Set the configuration
dev.set_configuration()

# Claim the interface
cfg = dev.get_active_configuration()
intf = cfg[(0,0)]

# Find the bulk OUT endpoint
ep = None
for ep_obj in intf:
    if usb.util.endpoint_direction(ep_obj.bEndpointAddress) == usb.util.ENDPOINT_OUT:
        ep = ep_obj
        break

# Send data to the TX pin to control the laser
def turn_off_laser():
    data = b'\x00'  # Low signal to turn off the laser
    dev.write(ep.bEndpointAddress, data, 1000)

def blink_laser():
    data_high = b'\xFF'  # High signal
    data_low = b'\x00'  # Low signal
    while True:
        dev.write(ep.bEndpointAddress, data_high, 1000)
        time.sleep(0.5)
        dev.write(ep.bEndpointAddress, data_low, 1000)
        time.sleep(0.5)

# Example usage:
turn_off_laser()
# or
blink_laser()

KeyboardInterrupt: 